### Initial Imports

In [5]:
import sys
sys.path.insert(0, 'C:/Users/enioh/Documents/Github/MMA-ML-Model')

import pandas as pd 
import numpy as np 

from os.path import abspath, dirname, join

### Dataset Loading

In [6]:
df = pd.read_csv(abspath(join(dirname(abspath('')), 'data/', 'processed/', 'extracted_stats.csv')))

In [3]:
# features = df.loc[:, df.columns.str.contains('precomp_([a-zA-Z_]+)_vs_opp', regex=True)]
# features.corr().abs()

C:\Users\enioh\AppData\Local\Temp\ipykernel_8312\2010163555.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features = df.loc[:, df.columns.str.contains('precomp_([a-zA-Z_]+)_vs_opp', regex=True)]


,precomp_height_prior_vs_opp,precomp_height_avg_vs_opp,precomp_height_windowavg_vs_opp,precomp_height_var_vs_opp,precomp_height_windowvar_vs_opp,precomp_height_peak_vs_opp,precomp_height_low_vs_opp,precomp_height_delta_vs_opp,precomp_reach_prior_vs_opp,precomp_reach_avg_vs_opp,...,precomp_ground_strikes_accuracy_pM_differential_low_vs_opp,precomp_ground_strikes_accuracy_pM_differential_delta_vs_opp,precomp_elo_differential_prior_vs_opp,precomp_elo_differential_avg_vs_opp,precomp_elo_differential_windowavg_vs_opp,precomp_elo_differential_var_vs_opp,precomp_elo_differential_windowvar_vs_opp,precomp_elo_differential_peak_vs_opp,precomp_elo_differential_low_vs_opp,precomp_elo_differential_delta_vs_opp
precomp_height_prior_vs_opp,1.000000,0.991778,0.992800,0.033894,0.037753,0.992599,0.988930,0.103579,0.633962,0.632132,...,0.020949,0.056326,0.047126,0.089313,0.069151,0.041364,0.012990,0.022214,0.056383,0.014663
precomp_height_avg_vs_opp,0.991778,1.000000,0.999453,0.069313,0.069992,0.997605,0.997208,0.012054,0.633616,0.638044,...,0.023518,0.086217,0.046882,0.088151,0.068604,0.040499,0.014258,0.027710,0.065600,0.013033
precomp_height_windowavg_vs_opp,0.992800,0.999453,1.000000,0.067472,0.068799,0.996807,0.996358,0.002566,0.634768,0.637838,...,0.023453,0.086217,0.046520,0.087913,0.068118,0.040493,0.014569,0.027485,0.065437,0.013410
precomp_height_var_vs_opp,0.033894,0.069313,0.067472,1.000000,0.994485,0.012476,0.142400,0.264879,0.027961,0.058389,...,0.000414,NaN,0.025050,0.037573,0.019104,0.032125,0.031708,0.001454,0.051015,0.018721
precomp_height_windowvar_vs_opp,0.037753,0.069992,0.068799,0.994485,1.000000,0.011581,0.142591,0.233104,0.031433,0.059092,...,0.000301,NaN,0.023596,0.035900,0.016331,0.031115,0.030499,0.002052,0.048220,0.018528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
precomp_elo_differential_var_vs_opp,0.041364,0.040499,0.040493,0.032125,0.031115,0.042192,0.037709,0.012654,0.056923,0.056187,...,0.035283,0.167577,0.041636,0.024300,0.032084,1.000000,0.560724,0.407798,0.518010,0.006448
precomp_elo_differential_windowvar_vs_opp,0.012990,0.014258,0.014569,0.031708,0.030499,0.012239,0.016697,0.010086,0.011486,0.010564,...,0.021407,0.216679,0.007322,0.008857,0.014288,0.560724,1.000000,0.281753,0.255806,0.004703
precomp_elo_differential_peak_vs_opp,0.022214,0.027710,0.027485,0.001454,0.002052,0.022802,0.021404,0.005490,0.034814,0.073109,...,0.099232,0.007126,0.495034,0.773048,0.653022,0.407798,0.281753,1.000000,0.194063,0.023362
precomp_elo_differential_low_vs_opp,0.056383,0.065600,0.065437,0.051015,0.048220,0.056752,0.052874,0.013543,0.080431,0.110034,...,0.143744,0.140744,0.423446,0.714133,0.541931,0.518010,0.255806,0.194063,1.000000,0.030577


Plenty of correlated features, reduce down some and begin some feature selection techniques

See feature_selection.py for functions on only choosing features that are correlated > 0.90

In [7]:
from src.models.XGBoost_Model import make_model
from src.models.train_model import train_model
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

clf = make_model()
clf, split_data = train_model(clf, df)

X_train, y_train, X_test, y_test = split_data

C:\Users/enioh/Documents/Github/MMA-ML-Model\src\models\train_model.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  post_comp_cols = df.loc[:, ~df.columns.str.contains('precomp_([a-zA-Z_]+)_vs_opp', regex=True)].columns.to_list() # All columns that do not have the precomp_ id or vs_opp id


Model Training Accuracy: 0.8171680799163666
Model Test Accuracy: 0.5877675840978593


In [8]:
def threshold_selector(clf, X_train, y_train, X_test, y_test):

    thresholds = np.arange(.1, 5, .05)
    avgs = []

    for thresh in thresholds:
        loopavg = []
        for count in range(0,1):
            selection = SelectFromModel(estimator=clf, threshold=f'{thresh}*median', prefit=True)
            select_X_train = selection.transform(X_train)

            model = make_model()
            model.fit(select_X_train, y_train)
            select_X_test = selection.transform(X_test)
            y_pred = model.predict(select_X_test)
            accuracy = accuracy_score(y_test, y_pred)

            msg = f"Thresh={thresh}, n={select_X_train.shape[1]}, Accuracy: {accuracy}"
            print(msg)
            loopavg.append(accuracy)
        avgs.append((thresh, np.mean(loopavg)))
    
    for avg in avgs:
        print(avg)

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
threshold_selector(clf, X_train, y_train, X_test, y_test)

Thresh=0.1, n=1310, Accuracy: 0.6
Thresh=0.15000000000000002, n=1310, Accuracy: 0.6
Thresh=0.20000000000000004, n=1310, Accuracy: 0.6
Thresh=0.25000000000000006, n=1310, Accuracy: 0.6
Thresh=0.30000000000000004, n=1309, Accuracy: 0.5853211009174312
Thresh=0.3500000000000001, n=1309, Accuracy: 0.5853211009174312
Thresh=0.40000000000000013, n=1308, Accuracy: 0.5889908256880734
Thresh=0.45000000000000007, n=1307, Accuracy: 0.5883792048929664
Thresh=0.5000000000000001, n=1306, Accuracy: 0.5908256880733945
Thresh=0.5500000000000002, n=1306, Accuracy: 0.5908256880733945
Thresh=0.6000000000000002, n=1302, Accuracy: 0.5920489296636086
Thresh=0.6500000000000001, n=1295, Accuracy: 0.581651376146789
Thresh=0.7000000000000002, n=1281, Accuracy: 0.5963302752293578
Thresh=0.7500000000000002, n=1268, Accuracy: 0.5957186544342508
Thresh=0.8000000000000002, n=1231, Accuracy: 0.582262996941896
Thresh=0.8500000000000002, n=1189, Accuracy: 0.5914373088685015
Thresh=0.9000000000000002, n=1113, Accuracy: 0.

KeyboardInterrupt: 

### 0.65 is best threshold it seems 